# Creating a database using yfinance stock data


## Getting the data


In [52]:
import sqlite3 as sql
import pandas as pd
import yfinance as yf

In [53]:


def get_cash_flow_increase(cash_flow):
      i=3
      old_cf = 0
      while i >-1:
            year = cash_flow.columns[i]

            cf = cash_flow[year].loc['Free Cash Flow']

            if float(cf) > old_cf:
                  old_cf = cf
            else:
                  return False
            
            i-=1
            
      return True



In [54]:
#Populate the database with the desired stock information



#top 50 stocks by market cap
stock_tickers = ["MSFT","CHGG", "AAPL", "NVDA", "GOOG", "GOOGL", "AMZN", "META",
                 "LLY", "AVGO", "TSLA", "V", "WMT", "XOM", "UNH", "MA", 
                 "ORCL", "COST", "ADBE", "NFLX", "CVX", "AMD", "KO", "ABBV", 
                 "MRK", "PEP", "TMO", "ACN", "CRM", "ABT", "LIN", "MCD", 
                 "MDT", "AMGN", "PYPL", "INTC", "HON", "RTX", "TXN", "QCOM", 
                 "UPS", "NKE", "PM", "SBUX", "IBM", "BMY", "PFE", "CAT", "SPGI", "DHR"
]


stock_info_dfs = []

for stock in stock_tickers:

    ticker = yf.Ticker(stock)

    balance_sheet = ticker.balance_sheet

    income_statement = ticker.income_stmt

    cash_flow = ticker.cash_flow

    ticker_list = []

    # categories = cash_flow.index.tolist()

    #displays all category names in the balance shet, income statement or cash flow report depending on which is entered
    # for category in categories:
    #     print(category)

    #data that will be used to determine the validity of a stock buy

    #create the database
    
    
    most_recent_year_b = balance_sheet.columns[0]
    most_recent_year_istmt = income_statement.columns[0]

    
    current_assets = balance_sheet.loc['Current Assets']
    current_liabilities = balance_sheet.loc['Current Liabilities']
    operating_income = income_statement.loc['Operating Income']
    total_revenue = income_statement.loc['Total Revenue']
    free_cash_flow_4 = cash_flow.loc['Free Cash Flow']

    data = {
        'Name': stock,
        'Total Current Assets': current_assets,
        'Total Current Liabilities': current_liabilities,
        'Operating Income': operating_income,
        'Total Revenue': total_revenue,
        'Free Cash Flow over last 4 years': free_cash_flow_4
    }

    # print(data)

   

    
    if float(current_assets[most_recent_year_b]/current_liabilities[most_recent_year_b]) > 1 and float(operating_income[most_recent_year_istmt]/total_revenue[most_recent_year_istmt]) > .15 and get_cash_flow_increase(cash_flow):
        data.update({'Buy': 1})
    else:
        data.update({'Buy': 0})
        
        



    data = pd.DataFrame(data)

    stock_info_dfs.append(data)




In [58]:
#for all the stock dataframes, add the stock ticker data to a csv file

stock_info = pd.concat(stock_info_dfs).dropna()
stock_info.index.name = 'Dates'
stock_info = stock_info.reset_index('Dates')
stock_info.to_csv('preprocessed_stock_info.csv')



## Train a model on the data

In [60]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier as dt

#load the data
stock_info = pd.read_csv('preprocessed_stock_info.csv')

X = stock_info.drop(['Name','Dates','Buy'], axis=1)
y = stock_info['Buy']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = dt()
model.fit(X_train, y_train)

print(model.score(X_test, y_test))



0.95
